<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EB%AC%B8%EC%98%81%EC%8B%9D/FORS_%EB%AC%B8%EC%98%81%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCM 파일 핸들링 코드 모음

참고 : https://noggame.tistory.com/15

## pcm을 wav로 변환하는 코드

PCM 데이터를 WAV로 변환하려는 경우 WAV 파일이 어떻게 쓰였는지 나타내는 Header가 필요하다. Header에는 samplerate나 소리 데이터의 길이 등의 정보가 들어가며, 직접 구현하기보다는 numpy, librosa, soundfile 등의 라이브러리를 사용하면 간단히 해결 가능하다.

In [ ]:
import numpy as np
import librosa as lr
import soundfile as sf

target = "/content/KsponSpeech_000001.pcm" # 타깃 파일 경로
destinationPath = "/content/KsponSpeech_000001.wav" #만들 파일 경로
buf = None

with open(target, 'rb') as tf:
    buf = tf.read()
    buf = buf+b'0' if len(buf)%2 else buf    # padding 0 (경우에 따라서 PCM 파일의 길이가 8bit[1byte]로 나누어 떨어지지 않는 경우가 있어 0으로 패딩값을 더해준다, 해당 처리를 하지 않는 경우 numpy나 librosa 라이브러리 사용 시 오류가 날 수 있다)

pcm_data = np.frombuffer(buf, dtype='int16')
wav_data = lr.util.buf_to_float(x=pcm_data, n_bytes=2)
sf.write(destinationPath, wav_data, 16000, format='WAV', endian='LITTLE', subtype='PCM_16')

## wav 파일을 pcm 으로 변환하는 코드

PCM과 WAV의 소리 데이터 부분은 동일하고, Header 부분만 차이가 나므로 WAV 파일에서 (Header를 제외한) 데이터 부분만 읽어서 파일로 만들면 PCM 파일에 해당한다.

In [ ]:
target = "{대상파일_경로}"
destinationPath = target[:-4]+".pcm" # {생성파일경로}
buf = None

with open(destinationPath, "wb") as d_file:
    t_file = open(target, "rb")
    t_bin = t_file.read()
    d_file.write(t_bin[44:])    # header 이외의 데이터를 pcm 파일로 저장
    t_file.close()

## pcm 파일 통합 하는 코드

PCM은 파일의 시작에 소리 정보를 담고있는 header가 존재하지 않기 때문에, raw 데이터(Byte Code)를 그대로 읽어서 합치면된다.

In [ ]:
targetList = ["{대상파일1_경로}", "{대상파일2_경로}"]
destinationPath = "{생성파일경로}"

buf = bytearray()
for file in targetList:
    f = open(file, 'rb')
    buf += f.read()
    f.close()

wf = open(destinationPath, 'wb')
wf.write(buf)
wf.close()



---



# OpenAI WhisperAI STT (오래걸림)

위스퍼 설치

In [ ]:
pip install openai-whisper

한국어 모델 다운로드하기

In [ ]:
!whisper-cli --model ko --download-dir .

음성 파일 변환하기

변환할 한국어 음성 파일을 준비합니다.
다음 Python 코드를 실행하여 음성 파일을 텍스트로 변환합니다:

In [ ]:
import whisper

# 모델 로드
model = whisper.load_model("small")


결과 확인하기

위 코드를 실행하면 음성 파일의 텍스트 인식 결과가 출력됩니다.
인식 결과는 result['text']에 저장됩니다.

OpenAI Whisper는 다양한 언어를 지원하며, 특히 한국어 모델의 성능이 우수합니다. 이를 활용하면 다양한 음성 기반 애플리케이션을 개발할 수 있습니다.

추가로, Whisper는 다음과 같은 특징이 있습니다:

지원 언어: 총 98개 언어 지원 (한국어 포함)

모델 크기: 작은 모델부터 대형 모델까지 다양한 옵션 제공

사용 요금: 무료로 사용 가능

---



In [ ]:

# 음성 파일 경로 설정
audio_file = "/content/KsponSpeech_000001.wav"

# 음성 파일 변환
result = model.transcribe(audio_file)
print(f"인식 결과: {result['text']}")



---



# NVIDIA NeMo STT (1초)

https://huggingface.co/eesungkim/stt_kr_conformer_transducer_large

모델을 훈련하고, 미세 조정하고, 플레이하려면 NVIDIA NeMo를 설치해야 합니다 . 최신 Pytorch 버전을 설치한 후 설치하는 것이 좋습니다.

In [ ]:
!pip install nemo_toolkit['all']

이 모델은 NeMo 툴킷[1](https://github.com/NVIDIA/NeMo)에서 사용할 수 있으며 추론을 위해 사전 훈련된 체크포인트로 사용하거나 다른 데이터 세트에 대한 미세 조정을 수행할 수 있습니다.


자동으로 모델 인스턴스화

In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("eesungkim/stt_kr_conformer_transducer_large")

음성파일 텍스트로 전환

In [ ]:
asr_model.transcribe(['KsponSpeech_000001.wav'])

많은 오디오 파일을 텍스트로 변환

In [ ]:
python [NEMO_GIT_FOLDER]/examples/asr/transcribe_speech.py  pretrained_name="eesungkim/stt_kr_conformer_transducer_large"  audio_dir="<DIRECTORY CONTAINING AUDIO FILES>"